In [3]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Define the data
patients = ['p1', 'p2', 'p3']
hospitals = ['h1', 'h2']

# Predicted wait times (WT), load imbalance costs (HL), and geographic proximity costs (GP)
WT = {
    ('p1', 'h1'): 30, ('p1', 'h2'): 20,
    ('p2', 'h1'): 40, ('p2', 'h2'): 25,
    ('p3', 'h1'): 35, ('p3', 'h2'): 30,
}

GP = {
    ('p1', 'h1'): 5, ('p1', 'h2'): 15,
    ('p2', 'h1'): 10, ('p2', 'h2'): 20,
    ('p3', 'h1'): 8, ('p3', 'h2'): 12,
}

# Current load (L_h) and capacity (C_h) of hospitals
L_h = {'h1': 50, 'h2': 60}
C_h = {'h1': 70, 'h2': 80}
average_load = sum(L_h.values()) / len(hospitals)

# Weighting factors
alpha = 1
beta = 1
gamma = 1

# Define the problem
problem = LpProblem("Patient_Hospital_Assignment", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(p, h) for p in patients for h in hospitals], cat="Binary")

# Objective function
problem += lpSum(
    alpha * WT[(p, h)] * x[(p, h)] +
    beta * abs(L_h[h] - average_load) / average_load * x[(p, h)] +
    gamma * GP[(p, h)] * x[(p, h)]
    for p in patients for h in hospitals
), "Total_Cost"

# Constraints
# Each patient is assigned to exactly one hospital
for p in patients:
    problem += lpSum(x[(p, h)] for h in hospitals) == 1, f"Assignment_{p}"

# Hospital capacity constraints
for h in hospitals:
    problem += lpSum(x[(p, h)] for p in patients) <= C_h[h], f"Capacity_{h}"

# Solve the problem
problem.solve()

# Display the results
print("Status:", problem.status)
for p in patients:
    for h in hospitals:
        if x[(p, h)].value() == 1:
            print(f"Patient {p} is assigned to hospital {h}.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/JEFFEYMARKUS/miniforge3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nb/5hgxnz3n2ls2gvdxrtqnclp80000gn/T/fb08ecd50bdf4fefa573224c41310dd2-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/nb/5hgxnz3n2ls2gvdxrtqnclp80000gn/T/fb08ecd50bdf4fefa573224c41310dd2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 122.273 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 122.273 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of whi